In [1]:
import warnings
import os   
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, year, month, countDistinct, avg, stddev, max, min, expr, asc, desc
warnings.filterwarnings("ignore")   

In [2]:
spark = SparkSession.builder \
    .appName("Spark SQL agg") \
    .config("spark.some.config.option", "some-value") \
    .enableHiveSupport() \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/30 15:46:51 INFO SparkEnv: Registering MapOutputTracker
24/05/30 15:46:51 INFO SparkEnv: Registering BlockManagerMaster
24/05/30 15:46:51 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/05/30 15:46:51 INFO SparkEnv: Registering OutputCommitCoordinator


In [3]:
df = spark.read.table('QQP')

df.show(5)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
24/05/30 15:47:19 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/05/30 15:47:34 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/05/30 15:47:49 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/05/30 15:48:04 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/05/30 15:48:19 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/05/30 15:48:34 WARN YarnSch

+---------------+--------------------+-------+--------------------+-----------------+-------+-------------+---------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+
|       PRODUCTO|        PRESENTACIÓN|  MARCA|           CATEGORÍA|         CATÁLOGO| PRECIO|FECHAREGISTRO|CADENACOMERCIAL|                GIRO|     NOMBRECOMERCIAL|           DIRECCIÓN|        ESTADO|     MUNICIPIO|  LATITUD|   LONGITUD|
+---------------+--------------------+-------+--------------------+-----------------+-------+-------------+---------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+
|BARRA DE SONIDO|HS205. BLUETOOTH,...|HISENSE|APARATOS ELECTRON...|ELECTRODOMESTICOS|1299.00|   2024-02-16|         COPPEL|TIENDA DE ELECTRO...|COPPEL SUCURSAL S...|CONVENCION NTE. 2...|AGUASCALIENTES|AGUASCALIENTES|21.896204|-102.309694|
|BARRA DE SONIDO|S40 Q. BLUETOOTH,...|     L

In [4]:
df_agg = df.withColumn(
    "canasta_basica",
    when(
        (col("PRODUCTO") == "ACEITE") & (col("PRESENTACIÓN") == "BOTELLA 946 ML. MIXTO"), 1
    ).when(
        (col("PRODUCTO") == "ARROZ") & (col("PRESENTACIÓN") == "BOLSA 1 KG. SUPER EXTRA"), 1
    ).when(
        (col("PRODUCTO") == "ATUN") & (col("PRESENTACIÓN") == "LATA 140 GR. EN HOJUELAS EN AGUA"), 1
    ).when(
        (col("PRODUCTO") == "AZUCAR") & (col("PRESENTACIÓN") == "BOLSA PLASTICO 2 KG. REFINADA"), 1
    ).when(
        (col("PRODUCTO") == "CARNE RES") & (col("PRESENTACIÓN") == "1 KG. GRANEL. FALDA O PARA DESHEBRAR"), 1
    ).when(
        (col("PRODUCTO") == "CEBOLLA") & (col("PRESENTACIÓN") == "1 KG. BLANCA SIN RABO"), 1
    ).when(
        (col("PRODUCTO") == "CHILE FRESCO") & (col("PRESENTACIÓN") == "1 KG. JALAPEÑO O CUARESMEÑO"), 1
    ).when(
        (col("PRODUCTO") == "CARNE CERDO") & (col("PRESENTACIÓN") == "1 KG. GRANEL. MILANESA O MILANESA DE PIERNA"), 1
    ).when(
        (col("PRODUCTO") == "FRIJOL") & (col("PRESENTACIÓN") == "BOLSA 1 KG. NEGRO"), 1
    ).when(
        (col("PRODUCTO") == "HUEVO") & (col("PRESENTACIÓN") == "PAQUETE CON 12 BLANCO"), 1
    ).when(
        (col("PRODUCTO") == "JABON DE TOCADOR") & (col("PRESENTACIÓN") == "BARRA 180 GR."), 1
    ).when(
        (col("PRODUCTO") == "JITOMATE") & (col("PRESENTACIÓN") == "1 KG. SALADETTE/ HUAJE O TOMATE SALADETTE/ HUAJE"), 1
    ).when(
        (col("PRODUCTO") == "LECHE PASTEURIZADA") & (col("PRESENTACIÓN") == "CAJA 1 LT."), 1
    ).when(
        (col("PRODUCTO") == "LIMON") & (col("PRESENTACIÓN") == "1 KG. AGRIO CON SEMILLA O LIMON COLIMA"), 1
    ).when(
        (col("PRODUCTO") == "MANZANA") & (col("PRESENTACIÓN") == "1 KG. GOLDEN"), 1
    ).when(
        (col("PRODUCTO") == "PIÑA") & (col("PRESENTACIÓN") == "1 KG. O PIÑA ESMERALDA O PIÑA CAYENA"), 1
    ).when(
        (col("PRODUCTO") == "PAN DE CAJA") & (col("PRESENTACIÓN") == "PAQUETE GRANDE 680 GR. REBANADO BLANCO"), 1
    ).when(
        (col("PRODUCTO") == "PAPA") & (col("PRESENTACIÓN") == "1 KG. ALFA/BLANCA"), 1
    ).when(
        (col("PRODUCTO") == "PAPEL HIGIENICO") & (col("PRESENTACIÓN") == "PAQUETE 12 ROLLOS. 200 HOJAS DOBLES"), 1
    ).when(
        (col("PRODUCTO") == "PASTA PARA SOPA") & (col("PRESENTACIÓN") == "PAQUETE 200 GR. SPAGHETTI"), 1
    ).when(
        (col("PRODUCTO") == "CARNE POLLO") & (col("PRESENTACIÓN") == "1 KG. GRANEL. PECHUGA CON HUESO O ANATÓMICA O SIN RABADILLA (CON PIEL)"), 1
    ).when(
        (col("PRODUCTO") == "SARDINA") & (col("PRESENTACIÓN") == "LATA 425 GR. EN TOMATE"), 1
    ).when(
        (col("PRODUCTO") == "TORTILLA DE MAIZ") & (col("PRESENTACIÓN") == "1 KG. GRANEL"), 1
    ).when(
        (col("PRODUCTO") == "ZANAHORIA") & (col("PRESENTACIÓN") == "1 KG. MEDIANA"), 1
    ).otherwise(0)
)

In [5]:
df_agg = df_agg.filter((col("canasta_basica") == 1) & (col("PRECIO").between(5, 1000)))

In [6]:
df_agg = df_agg.groupBy(
    col("FECHAREGISTRO").alias("fecha"),
    year(col("FECHAREGISTRO")).alias("fecha_year"),
    month(col("FECHAREGISTRO")).alias("fecha_month"),
    col("PRODUCTO").alias("producto"),
    col("PRESENTACIÓN").alias("descripcion")
).agg(
    countDistinct("MARCA").alias("count_marca"),
    countDistinct("NOMBRECOMERCIAL").alias("count_tiendas"),
    countDistinct("ESTADO").alias("count_estados"),
    countDistinct("MUNICIPIO").alias("count_municipios"),
    avg("PRECIO").alias("precio_avg"),
    stddev("PRECIO").alias("precio_desv_std"),
    expr("percentile_approx(PRECIO, 0.5)").alias("precio_median"),
    max("PRECIO").alias("precio_max"),
    min("PRECIO").alias("precio_min")
)

In [7]:
df_agg.show()

+----------+----------+-----------+------------------+--------------------+-----------+-------------+-------------+----------------+----------+------------------+-------------+----------+----------+
|     fecha|fecha_year|fecha_month|          producto|         descripcion|count_marca|count_tiendas|count_estados|count_municipios|precio_avg|   precio_desv_std|precio_median|precio_max|precio_min|
+----------+----------+-----------+------------------+--------------------+-----------+-------------+-------------+----------------+----------+------------------+-------------+----------+----------+
|2015-01-09|      2015|          1|            AZUCAR|BOLSA PLASTICO 2 ...|          1|           34|           22|              29| 33.829412| 3.776986084951598|        35.00|     38.50|     24.50|
|2015-02-06|      2015|          2|           MANZANA|        1 KG. GOLDEN|          1|           55|           28|              40| 26.500000| 5.792795653350935|        25.95|     40.00|     15.00|
|2015

In [8]:
df_agg.count()

51733

In [9]:
value_counts = df_agg.groupBy("producto").count()
value_counts_sorted = value_counts.orderBy(desc("count"))
value_counts_sorted.show(25)

+------------------+-----+
|          producto|count|
+------------------+-----+
|  TORTILLA DE MAIZ| 2347|
|           SARDINA| 2344|
|          JITOMATE| 2342|
|       PAN DE CAJA| 2342|
|              PAPA| 2342|
|           CEBOLLA| 2341|
|             LIMON| 2341|
|       CARNE CERDO| 2341|
|   PASTA PARA SOPA| 2341|
|             ARROZ| 2341|
|       CARNE POLLO| 2341|
|      CHILE FRESCO| 2341|
|           MANZANA| 2339|
|              ATUN| 2338|
|            FRIJOL| 2338|
|         ZANAHORIA| 2338|
|LECHE PASTEURIZADA| 2336|
|   PAPEL HIGIENICO| 2334|
|         CARNE RES| 2169|
|             HUEVO| 2084|
|            AZUCAR| 1880|
|            ACEITE| 1532|
|              PIÑA| 1294|
|  JABON DE TOCADOR|  647|
+------------------+-----+



In [10]:
# df_agg.coalesce(1).write.mode('overwrite').parquet('/user/spark/qqp_agg')

In [11]:
# !hdfs dfs -copyToLocal /user/spark/qqp_agg /qqp_agg

In [14]:
jdbc_url = "jdbc:mysql://34.132.226.181:3306/QQP"
properties = {
    "user": "big_data_user",
    "password": "Example123"
}

df_agg.write.jdbc(url=jdbc_url, table="aggregated", mode="overwrite", properties=properties)

In [15]:
spark.stop()